In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score

# 다항분포 나이브베이즈를 위한 라이브러리를 임포트.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

np.random.seed(5)

# 문제 정의 
* 영화 리뷰에 다항분포 나이브베이즈 분류를 활용해 영화 리뷰가 긍정인지 부정적인지를 분류해 보겠습니다.

# 데이터 수집

In [2]:
#데이터의 표현과 최종적으로 이끌어낼 데이터의 타입에 따라서 다항분포 나이브베이즈와 베르누이 나이브베이즈로 나뉘고 판단하면 됨
#다항분포 나이브베이즈에서 데이터를 어떻게 체크할까? 
#지금의 감상평은 긍정적인 경우, 부정적인 경우에 그 단어에 대한 빈도수까지를 고려해서 
#긍정에 많이 등장하는 단어, 부정에 많이 등장하는 단어들로써 학습시킨다고 했을 때 
#긍정과 부정에 가중치가 더 적용되어지게끔 하면서 정확도를 높여갈 수 있지 않을까하는 것임.
#등장하는 단어에 갯수까지 의미를 부여해서 학습을 시키는 것!!
#==>스팸에 등장하는 단어는 일반단어나 스팸단어에서 기본적으로 등장하는 단어로 나뉘어질 수 없음.

review_list = [ # 학습 데이터
                {'movie_review': 'this is great great movie. I will watch again', 'type': 'positive'},
                {'movie_review': 'I like this movie', 'type': 'positive'},
                {'movie_review': 'amazing movie in this year', 'type': 'positive'},
                {'movie_review': 'cool my boyfriend also said the movie is cool', 'type': 'positive'},
                {'movie_review': 'awesome of the awesome movie ever', 'type': 'positive'},
                {'movie_review': 'shame I wasted money and time', 'type': 'negative'},
                {'movie_review': 'regret on this move. I will never never what movie from this director', 'type': 'negative'},
                {'movie_review': 'I do not like this movie', 'type': 'negative'},
                {'movie_review': 'I do not like actors in this movie', 'type': 'negative'},
                {'movie_review': 'boring boring sleeping movie', 'type': 'negative'}
             ] 

df = pd.DataFrame(review_list)
df 

,movie_review,type
0,this is great great movie. I will watch again,positive
1,I like this movie,positive
2,amazing movie in this year,positive
3,cool my boyfriend also said the movie is cool,positive
4,awesome of the awesome movie ever,positive
5,shame I wasted money and time,negative
6,regret on this move. I will never never what m...,negative
7,I do not like this movie,negative
8,I do not like actors in this movie,negative
9,boring boring sleeping movie,negative


In [3]:
df['label'] = df['type'].map({"positive":1, "negative":0})
df

,movie_review,type,label
0,this is great great movie. I will watch again,positive,1
1,I like this movie,positive,1
2,amazing movie in this year,positive,1
3,cool my boyfriend also said the movie is cool,positive,1
4,awesome of the awesome movie ever,positive,1
5,shame I wasted money and time,negative,0
6,regret on this move. I will never never what m...,negative,0
7,I do not like this movie,negative,0
8,I do not like actors in this movie,negative,0
9,boring boring sleeping movie,negative,0


In [4]:
df_x = df['movie_review']
df_y = df['label']

In [5]:
cv = CountVectorizer() #숫자로 변환시 활용 -> 이전에는 binary를 true로 해줬었음. (표시해달라는 의미)
                       #default로 냅두면 숫자로 변환하면서 누적을 해주는 것임. 
x_traincv = cv.fit_transform(df_x)

In [6]:
cv.get_feature_names_out() #현재 등장한 모든 단어를 알파벳순서로 매핑한 것을 반환해줌 37딘어

array(['actors', 'again', 'also', 'amazing', 'and', 'awesome', 'boring',
       'boyfriend', 'cool', 'director', 'do', 'ever', 'from', 'great',
       'in', 'is', 'like', 'money', 'move', 'movie', 'my', 'never', 'not',
       'of', 'on', 'regret', 'said', 'shame', 'sleeping', 'the', 'this',
       'time', 'wasted', 'watch', 'what', 'will', 'year'], dtype=object)

In [7]:
#반환되어진 실제 데이터를 확인
encoded_input = x_traincv.toarray()
encoded_input #단어가 등장한 횟수에 따라서 매핑이 되고 있음

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 2,
        0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0

In [8]:
#기존 문장으로 다시 복원
cv.inverse_transform(encoded_input[[0]]) #주의) 리스트자료형으로 넣어줘야 한다.

[array(['again', 'great', 'is', 'movie', 'this', 'watch', 'will'],
       dtype='<U9')]

## 다항 분포 나이브베이즈 분류

In [9]:
# 기존의 데이터로 학습을 진행.
mnb = MultinomialNB()

y_train = df_y.astype('int') #int형으로 변환되어져 있을지라도 다시 변환을 꼭 해주도록 하자.
mnb.fit(x_traincv, y_train)

MultinomialNB()

In [12]:
test_feedback_list = [  #테스트 데이터
                {'movie_review': 'great great great movie ever', 'type': 'positive'},
                {'movie_review': 'I like this amazing movie', 'type': 'positive'},
                {'movie_review': 'my boyfriend said great movie ever', 'type': 'positive'},
                {'movie_review': 'cool cool cool', 'type': 'positive'},
                {'movie_review': 'awesome boyfriend said cool movie ever', 'type': 'positive'},
                {'movie_review': 'shame shame shame', 'type': 'negative'},
                {'movie_review': 'awesome director shame movie boring movie', 'type': 'negative'},
                {'movie_review': 'do not like this movie', 'type': 'negative'},
                {'movie_review': 'I do not like this boring movie', 'type': 'negative'},
                {'movie_review': 'aweful terrible boring movie', 'type': 'negative'}
             ] 

test_df = pd.DataFrame(test_feedback_list)
test_df['label'] = test_df['type'].map({'positive':1, 'negative':0})

#훈련데이터와 정답데이터로 나누어서 변수에 담아주도록 한다.
test_x = test_df['movie_review']
test_y = test_df['label']

x_testcv = cv.transform(test_x) #***훈련을 시키면 안됨!! 기존에 반영한 그대로 사용을 해야함!!

# 테스트

In [13]:
predicted = mnb.predict(x_testcv) 

# 정확도(accuracy)

In [14]:
accuracy_score(test_y, predicted)

1.0